In [1]:
import pandas as pd
import numpy as np
import json
import ast

In [2]:
train = pd.read_csv('data/train.csv')
train.tail(3)

,date,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
1213,20210428,"[{""engagementMetricsDate"":""2021-04-29"",""player...","[{""gamePk"":634309,""gameType"":""R"",""season"":2021...","[{""playerId"":429722,""gameDate"":""2021-04-28"",""t...","[{""home"":1,""gamePk"":634310,""gameDate"":""2021-04...","[{""home"":0,""teamId"":111,""gamePk"":634310,""gameD...","[{""transactionId"":480728,""playerId"":545358,""pl...","[{""season"":2021,""gameDate"":""2021-04-28"",""divis...",NaN,"[{""gamePk"":634317,""gameDate"":""2021-04-28"",""gam...",NaN,NaN
1214,20210429,"[{""engagementMetricsDate"":""2021-04-30"",""player...","[{""gamePk"":634330,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-29"",""t...","[{""home"":1,""gamePk"":634330,""gameDate"":""2021-04...","[{""home"":0,""teamId"":119,""gamePk"":634346,""gameD...","[{""transactionId"":480993,""playerId"":606965,""pl...","[{""season"":2021,""gameDate"":""2021-04-29"",""divis...",NaN,"[{""gamePk"":634346,""gameDate"":""2021-04-29"",""gam...",NaN,NaN
1215,20210430,"[{""engagementMetricsDate"":""2021-05-01"",""player...","[{""gamePk"":634287,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-30"",""t...","[{""home"":1,""gamePk"":634305,""gameDate"":""2021-04...","[{""home"":1,""teamId"":135,""gamePk"":634303,""gameD...",NaN,"[{""season"":2021,""gameDate"":""2021-04-30"",""divis...","[{""awardId"":""NLRRELMON"",""awardName"":""NL Reliev...","[{""gamePk"":634327,""gameDate"":""2021-04-30"",""gam...",NaN,NaN


In [159]:
# !pip install pandarallel 

import gc

import numpy as np
import pandas as pd
from pathlib import Path

# from pandarallel import pandarallel
# multiprocessing.set_start_method("fork", force=True)
# pandarallel.initialize()

train = pd.read_csv('data/train.csv')

null = np.nan
true = True
false = False

for col in train.columns:
    print(col)
    if col == 'date': continue

    _index = train[col].notnull()
    train.loc[_index, col] = train.loc[_index, col].apply(lambda x: eval(x))

    outputs = []
    for index, date, record in train.loc[_index, ['date', col]].itertuples():
        _df = pd.DataFrame(record)
        _df['index'] = index
        _df['date'] = date
        outputs.append(_df)

    outputs = pd.concat(outputs).reset_index(drop=True)

    outputs.to_csv(f'{col}_train.csv', index=False)
    outputs.to_pickle(f'{col}_train.pkl')

    del outputs
    del train[col]
    gc.collect()

date
nextDayPlayerEngagement
games
rosters
playerBoxScores
teamBoxScores
transactions
standings
awards
events
playerTwitterFollowers
teamTwitterFollowers


In [3]:
def parse_json(column_to_parse):
    for col in column_to_parse:
        print(f"Parse columns {col}")
        target_df = pd.DataFrame()
        target_df['date'] = train['date']
        for i in range(train.shape[0]):
            cell = train.iloc[i][col]
            if cell is not None:
                # If a cell is not None make list
                try:
                    value = json.loads(cell)
                except ValueError:
                    continue
                for k, v in enumerate(value):
                    prefix_col = f'{col}_{k}'

                    for key, value in v.items():
                        new_col = f'{prefix_col}_{key}'
                        if new_col not in target_df.columns:
                            train[new_col] = None
                        target_df.at[i, new_col] = value
            # Save to csv
        target_df.to_csv('data/'+ col + '.csv')
        print(target_df.shape)
    return target_df.head(5)